### Необходимые импорты

In [10]:
from datasets import load_dataset

import pandas as pd
import re
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, make_scorer, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Загрузка данных и анализ

In [3]:
ds = load_dataset("stanfordnlp/imdb")

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
first_3 = ds["train"].select(range(3))

# Вывод данных
for example in first_3:
    print("Текст:", example["text"])
    print("Метка:", example["label"])
    print("-" * 150)

Текст: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betwe

### Разделение на выборки

In [6]:
train_texts = ds["train"]["text"]
y_train = ds["train"]["label"]
test_texts = ds["test"]["text"]
y_test = ds["test"]["label"]

### Токенизация и лемматизация

In [7]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text) 
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return ' '.join(words)

train_texts_clean = [clean_text(t) for t in train_texts]
test_texts_clean = [clean_text(t) for t in test_texts]

In [8]:
stop_words = stopwords.words('english')

### Модели

#### Пайплайн для логистической регрессии

In [11]:
pipeline_lr = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stop_words, max_features=10000)),
    ('model', LogisticRegression(max_iter=1000))
])

param_lr = {'model__C': [0.01, 0.1, 1, 10]}

#### Пайплайн для дерева решений

In [12]:
pipeline_dt = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stop_words, max_features=10000)),
    ('model', DecisionTreeClassifier())
])

param_dt = {
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': [2, 5]
}

#### Пайплайн для градиентного бустинга

In [13]:
pipeline_cb = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stop_words, max_features=10000)),
    ('model', CatBoostClassifier(task_type='GPU', devices='0', verbose=0, gpu_ram_part=0.7))
])

### gridsearch

In [14]:
f1 = make_scorer(f1_score, average='binary', pos_label=1)

In [15]:
grid_lr = GridSearchCV(
    pipeline_lr,
    param_grid=param_lr,
    scoring=f1,
    cv=5,
    n_jobs=-1
)

grid_dt = GridSearchCV(
    pipeline_dt,
    param_grid=param_dt,
    scoring=f1,
    cv=5,
    n_jobs=-1
)

### Обучение

In [ ]:
grid_lr.fit(train_texts_clean, y_train)
grid_dt.fit(train_texts_clean, y_train)
pipeline_cb.fit(np.array(train_texts_clean), np.array(y_train))

In [ ]:
y_pred_lr = grid_lr.best_estimator_.predict(test_texts_clean)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

y_pred_dt = grid_dt.best_estimator_.predict(test_texts_clean)
accuracy_dt = accuracy_score(y_test, y_pred_dt)

y_pred_cb = pipeline_cb.predict(train_texts_clean)
accuracy_cb = accuracy_score(y_train, y_pred_cb)
f1_cb = f1_score(np.array(y_train), y_pred_cb, average='binary', pos_label=1)

print(f"лучший f1 логистической модели: {grid_lr.best_score_:.4f}")
print(f"Лучший f1 дерева решений : {grid_dt.best_score_:.4f}")
print(f"Лучший f1 градиентного бустинга : {f1_cb:.4f}")

In [ ]:
print(f"Accuracy логистической регрессии: {accuracy_lr:.4f}")
print(f"Accuracy дерева решений: {accuracy_dt:.4f}")
print(f"Accuracy градиентного бустинга: {accuracy_cb:.4f}")